In [0]:
# === CELL 0: NUCLEAR AUTH FIX (RUN THIS FIRST) ===
spark.conf.set(
    "fs.azure.account.key.goodreads60104699.dfs.core.windows.net",
    "slStfm+E+Qmq6wT2Yzk4kxaXe16kLK5e+erlt72LzutDZGHdWjDkvMYebsSBOnV2JH0NCnIjLj40+AStdSiXVA=="
)

# Verify connection — THIS WILL WORK
display(dbutils.fs.ls("abfss://lakehouse@goodreads60104699.dfs.core.windows.net/"))
print("AUTH FIXED — YOU ARE A GOD")

path name size modificationTime abfss://lakehouse@goodreads60104699.dfs.core.windows.net/bronze/ bronze/ 0 1762769150000 abfss://lakehouse@goodreads60104699.dfs.core.windows.net/gold/ gold/ 0 1762186237000 abfss://lakehouse@goodreads60104699.dfs.core.windows.net/processed/ processed/ 0 1762183898000 abfss://lakehouse@goodreads60104699.dfs.core.windows.net/raw/ raw/ 0 0 abfss://lakehouse@goodreads60104699.dfs.core.windows.net/silver/ silver/ 0 1762158061000

AUTH FIXED — YOU ARE A GOD


In [0]:
spark.conf.set(
    "fs.azure.account.key.goodreads60104699.dfs.core.windows.net",
    "slStfm+E+Qmq6wT2Yzk4kxaXe16kLK5e+erlt72LzutDZGHdWjDkvMYebsSBOnV2JH0NCnIjLj40+AStdSiXVA=="
)

In [0]:
from pyspark.dbutils import DBUtils
dbutils = DBUtils(spark)

# STEP 3 — List all folders in your Lakehouse root
display(dbutils.fs.ls("abfss://lakehouse@goodreads60104699.dfs.core.windows.net/"))

# STEP 4 — List GOLD layer (for verification)
display(dbutils.fs.ls("abfss://lakehouse@goodreads60104699.dfs.core.windows.net/gold/"))

# STEP 5 — (Optional) List PROCESSED layer
display(dbutils.fs.ls("abfss://lakehouse@goodreads60104699.dfs.core.windows.net/processed/"))


path name size modificationTime abfss://lakehouse@goodreads60104699.dfs.core.windows.net/bronze/ bronze/ 0 1762769150000 abfss://lakehouse@goodreads60104699.dfs.core.windows.net/gold/ gold/ 0 1762186237000 abfss://lakehouse@goodreads60104699.dfs.core.windows.net/processed/ processed/ 0 1762183898000 abfss://lakehouse@goodreads60104699.dfs.core.windows.net/raw/ raw/ 0 0 abfss://lakehouse@goodreads60104699.dfs.core.windows.net/silver/ silver/ 0 1762158061000

path name size modificationTime abfss://lakehouse@goodreads60104699.dfs.core.windows.net/gold/curated_reviews/ curated_reviews/ 0 1762866463000

path name size modificationTime abfss://lakehouse@goodreads60104699.dfs.core.windows.net/processed/reviews/ reviews/ 0 1762184228000 abfss://lakehouse@goodreads60104699.dfs.core.windows.net/processed/reviews_clean/ reviews_clean/ 0 1762183898000 abfss://lakehouse@goodreads60104699.dfs.core.windows.net/processed/reviews_clean_final/ reviews_clean_final/ 0 1762844898000 abfss://lakehouse@goodreads60104699.dfs.core.windows.net/processed/reviews_clean_v2/ reviews_clean_v2/ 0 1762866273000

In [0]:
# ==========================================================
# STEP 2 — READ SILVER LAYER PARQUET FILES  (CORRECT PATHS)
# ==========================================================

books = spark.read.parquet("abfss://lakehouse@goodreads60104699.dfs.core.windows.net/silver/books/")
authors = spark.read.parquet("abfss://lakehouse@goodreads60104699.dfs.core.windows.net/silver/authors/")
reviews = spark.read.parquet("abfss://lakehouse@goodreads60104699.dfs.core.windows.net/silver/reviews/")

# Show schemas for verification
books.printSchema()
authors.printSchema()
reviews.printSchema()

# Preview sample data
books.show(5, truncate=False)
authors.show(5, truncate=False)
reviews.show(5, truncate=False)


root
 |-- book_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- average_rating: double (nullable = true)

root
 |-- name: string (nullable = true)
 |-- author_id: string (nullable = true)

root
 |-- book_id: string (nullable = true)
 |-- date_added: timestamp (nullable = true)
 |-- date_updated: string (nullable = true)
 |-- n_comments: long (nullable = true)
 |-- n_votes: long (nullable = true)
 |-- rating: integer (nullable = true)
 |-- read_at: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- review_text: string (nullable = true)
 |-- started_at: string (nullable = true)
 |-- user_id: string (nullable = true)

+--------+----------------------------------------------+-----------------------------------------------+--------------+
|book_id |title                                         |authors                                        |average_rating|
+--------+--------------------------------------------

In [0]:
# ==========================================================
# STEP 3 — CLEAN THE REVIEWS DATASET (REBUILD)
# ==========================================================

from pyspark.sql.functions import col, trim, length

# 1️⃣ Filter valid reviews — make sure key IDs exist
df = reviews.filter(
    col("review_id").isNotNull() &
    col("book_id").isNotNull() &
    col("user_id").isNotNull()
)

# 2️⃣ Keep only ratings between 1 and 5
df = df.withColumn("rating_int", col("rating").cast("int")).filter(
    (col("rating_int").isNotNull()) &
    (col("rating_int") >= 1) &
    (col("rating_int") <= 5)
)

# 3️⃣ Keep only non-empty reviews (≥ 10 characters)
df = df.withColumn("review_text", trim(col("review_text"))).filter(
    col("review_text").isNotNull() &
    (length(col("review_text")) >= 10)
)

# 4️⃣ Remove duplicate review IDs
df = df.dropDuplicates(["review_id"])

# 5️⃣ Final cleaned dataset
reviews_clean = df.select(
    "review_id", "book_id", "user_id",
    col("rating_int").alias("rating"),
    "review_text", "date_added"
)

# 6️⃣ Check result
reviews_clean.show(5, truncate=False)
print("✅ Total Cleaned Reviews:", reviews_clean.count())


+--------------------------------+--------+--------------------------------+------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
# ==========================================================
# STEP 4 — SAVE CLEANED REVIEWS TO PROCESSED LAYER
# ==========================================================

output_path = "abfss://lakehouse@goodreads60104699.dfs.core.windows.net/processed/reviews_clean/"

# Overwrite mode ensures old data is replaced with the new cleaned data
reviews_clean.write.mode("overwrite").parquet(output_path)

print("✅ Cleaned reviews saved successfully to:", output_path)


✅ Cleaned reviews saved successfully to: abfss://lakehouse@goodreads60104699.dfs.core.windows.net/processed/reviews_clean/


In [0]:
display(dbutils.fs.ls("abfss://lakehouse@goodreads60104699.dfs.core.windows.net/processed/"))


path name size modificationTime abfss://lakehouse@goodreads60104699.dfs.core.windows.net/processed/reviews/ reviews/ 0 1762184228000 abfss://lakehouse@goodreads60104699.dfs.core.windows.net/processed/reviews_clean/ reviews_clean/ 0 1762183898000 abfss://lakehouse@goodreads60104699.dfs.core.windows.net/processed/reviews_clean_final/ reviews_clean_final/ 0 1762844898000 abfss://lakehouse@goodreads60104699.dfs.core.windows.net/processed/reviews_clean_v2/ reviews_clean_v2/ 0 1762866273000

In [0]:
storage_account = "goodreads60104699"

# PROCESSED (PARQUET) — THESE EXIST!
books_path    = f"abfs://files@{storage_account}.dfs.core.windows.net/processed/books"
reviews_path  = f"abfs://files@{storage_account}.dfs.core.windows.net/processed/reviews"
authors_path  = f"abfs://files@{storage_account}.dfs.core.windows.net/processed/authors"

# RAW (JSON) — ONLY REVIEWS & AUTHORS
reviews_raw_path = f"abfs://files@{storage_account}.dfs.core.windows.net/raw/reviews/goodreads_reviews_dedup.json"
authors_raw_path = f"abfs://files@{storage_account}.dfs.core.windows.net/raw/authors/"

print("PATHS FIXED — BOOKS FROM PROCESSED!")

PATHS FIXED — BOOKS FROM PROCESSED!


In [0]:
# === LOAD RAW JSON (reviews & authors) ===
reviews_raw_df = spark.read.json(reviews_raw_path)
authors_raw_df = spark.read.json(authors_raw_path)

# === LOAD PARQUET (books, reviews, authors) ===
books_df    = spark.read.parquet(books_path)
reviews_df  = spark.read.parquet(reviews_path)
authors_df  = spark.read.parquet(authors_path)

print("ALL DATA LOADED CORRECTLY!")

ALL DATA LOADED CORRECTLY!


In [0]:
# === CELL 1: FINAL PATHS (NO DOT, NO SLASH) ===
storage_account = "goodreads60104699"

books_path    = f"abfs://files@{storage_account}.dfs.core.windows.net/processed/books"
reviews_path  = f"abfs://files@{storage_account}.dfs.core.windows.net/processed/reviews"
authors_path  = f"abfs://files@{storage_account}.dfs.core.windows.net/processed/authors"

print("FINAL PATHS (NO DOT):")
print(books_path)

FINAL PATHS (NO DOT):
abfs://files@goodreads60104699.dfs.core.windows.net/processed/books


In [0]:
# === CELL 2: SET KEY ===
spark.conf.set(
    "fs.azure.account.key.goodreads60104699.dfs.core.windows.net",
    "1Y6DR3385TRqNYqEC3YadyFEpnrq0gNyUA0QVIx3CYqJ63V5nRpNrECewWQR9gdq8jVxaIynKp08+AStoPO4Ew=="  # ← Paste from Storage → Access keys
)

In [0]:
# === CELL 3: RELOAD PARQUET ===
books_df    = spark.read.parquet(books_path)
reviews_df  = spark.read.parquet(reviews_path)
authors_df  = spark.read.parquet(authors_path)

print("PARQUET LOADED — NO ERRORS!")

PARQUET LOADED — NO ERRORS!


In [0]:
# === CELL 5: Read RAW JSON with PERMISSIVE mode (for bad rows) ===
raw_reviews_path = f"abfs://lakehouse@{storage_account}.dfs.core.windows.net/raw/reviews/goodreads_reviews_dedup.json"

reviews_raw = spark.read \
    .option("mode", "PERMISSIVE") \
    .option("columnNameOfCorruptRecord", "_corrupt_record") \
    .json(raw_reviews_path)

print("Raw reviews with corrupt records loaded!")
reviews_raw.show(5, truncate=False)

Raw reviews with corrupt records loaded!
+--------+------------------------------+------------------------------+----------+-------+------+------------------------------+--------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
# ✅ Verify Gold Delta table
check_df = spark.read.format("delta").load("abfss://lakehouse@goodreads60104699.dfs.core.windows.net/gold/curated_reviews/")
display(check_df.limit(10))


review_id book_id title author_id name user_id rating review_text date_added average_rating bd094abe01c19ae84b8f5d41b70acb34 21901563 In This Life (Past Life Series #1) 34360337853 {6479371 2598fc0f8c5b9b8d2cf496f54784eb8f 5 Have you ever had a moment of deja vu? Most people just shake it off and come up with a thousand reasons why they felt that way; but has it ever made you wonder if there's more to it? Scientific studies will all give conflicting information about why people experience this; anxiety, stress, mental illness... there are literally hundreds of explanations out there; which do you believe? Do you believe in past lives and reincarnation? That sometimes we just don't get it right the first time around so we are given a second chance? In This Life is the story of Dr. Lottie Morgan. She begins to experience intense episodes of deja vu; except to her it isn't as simple as feeling as though it's happened before, she is actually recalling memories of a time that is not the present. This is an excellent story. I loved the writing style and how seamlessly the story flows between Lottie's recalled memories and the present. The characters are developed as the story progresses and as we pass between the past and the present we gain further understanding as to why the characters behave the way they do. The only negative I could pick out of the whole book was that there were times where I felt like Lottie didn't have enough backbone. She tried to always handle every situation as a psychologist and not as a person (I'm not even sure that's making sense)... she came across as a strong character, but there were times when her reactions didn't reflect that. At first I didn't like David very much. He was a little too controlling and, often times, demanding for my liking. In the beginning I got the feeling that he felt like Lottie was "his property" (he was not abusive or anything like that); but as the story progresses we do gain a better understanding of his actions. Galen is a likeable character, but I have a feeling there's so much more to him than meets the eye. Paul is likeable to a point... you'll understand what I mean when you get there. The plot twist, while kind of predictable, wasn't exactly what I was expecting. This was a relatively quick read; mainly because the story does flow so seamlessly, it's hard to put it down once you get going. I truly can't imagine holding a grudge for a thousand years (I have a hard time staying mad at my husband for more than 5 minutes!) and waiting out an entire lifetime to exact revenge! But that's exactly what happens. The intrigue of Ancient Egypt, past lives, second chances, falling in love all over again, mystery, suspense.... there's a little bit of everything in here. I highly recommend this book. And I have it on good authority that Covet, the second book in the series, will be re-released soon for everyone's reading pleasure. 2014-08-13T03:39:50Z 4.09 b97c83e0f682324b07d72cbf274a12bb 366300 Walter the Farting Dog 34360337882 } 84434593795d4d1cffb69e642b98e078 5 loved it. hilarious children's book 2014-04-26T19:01:35Z 3.88 4018edede123694adce9fb68dde74043 3200179 The Year's Best Horror Stories XV 34360338409 Contributor} 027b868702def7d32a349bff3deefd5a 1 On page 172 of 300- the first story is my favorite. The rest have been boring, endings really pathetically written. I personally wouldn't recommend this book. I love horror stories/movies but this isn't scary, freaky or entertaining. Disappointed . 2014-09-06T21:50:10Z 3.89 5fee02e433bb4d924de1cf48c1d8e9d9 13585708 My Teacher 34360338793 Illustrations} 2444d15de119591ec47b12af3d6bb279 5 I really liked this. It was a great book for the night before the first day of kindergarten at our house. I saw the city schools I went to in parts of the book. It had racial and economic diversity, something that can be tough to find in kid's books. 2014-08-12T23:58:48Z 3.99 d571d4a9c5da78a9a43976154ebd38f7 35820364 Beauty Sick: How the Cultura